In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, assemble
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.tools.visualization import circuit_drawer, array_to_latex
from qiskit.quantum_info.operators import Operator
import numpy as np
svsim = Aer.get_backend('aer_simulator')
simulator = QasmSimulator()

In [2]:
#Matriz para f:{0,1}^2 -> {0,1}, donde f(|00>)=1,f(|01>)=1,f(|10>)=0,f(|11>)=0.
#Pag 181, Noson S Yanofsky and Mirco A Mannucci. Quantum computing for computer scientists

f=Operator([[0,1,0,0,0,0,0,0],
            [1,0,0,0,0,0,0,0],
            [0,0,0,1,0,0,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,0,0,1,0,0],
            [0,0,0,0,0,0,1,0],
            [0,0,0,0,0,0,0,1]])

#Se va a seguir el mismo esquema al principio, para comprobar que los statevector van al revés de los esperado
#|abc> -> |cba>

#Se comprueba el statevector antes de f y después de f para todas las posibilidades

In [3]:
#Comprobamos para |000>

q=QuantumCircuit(3)

q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

#De aquí obtenemos que el primer elemento es |000>

<IPython.core.display.Latex object>

In [4]:
q=QuantumCircuit(3,2)

q.append(f,[0,1,2])

q.draw()

┌──────────┐
q_0: ┤0         ├
     │          │
q_1: ┤1 Unitary ├
     │          │
q_2: ┤2         ├
     └──────────┘
c: 2/════════════

In [5]:
q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

#Tras aplicar f a |000>, debería ir a |001>, por lo que suponemos que el segundo elemento es |001>

<IPython.core.display.Latex object>

In [6]:
#Comprobamos para |100>

q=QuantumCircuit(3)

q.x(0)

q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

#Primera contradicción, |100> es el segundo elemento de la base, pero según lo anterior, era el |001>

<IPython.core.display.Latex object>

In [7]:
q=QuantumCircuit(3)

q.x(0)

q.append(f,[0,1,2])

q.draw()

┌───┐┌──────────┐
q_0: ┤ X ├┤0         ├
     └───┘│          │
q_1: ─────┤1 Unitary ├
          │          │
q_2: ─────┤2         ├
          └──────────┘

In [8]:
q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

# De |100> deberíamos obtener |100>, pero en cambio nos devuelve el primer elemento, que era |000>.

<IPython.core.display.Latex object>

In [9]:
#Comprobamos para |010>

q=QuantumCircuit(3)

q.x(1)

q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

<IPython.core.display.Latex object>

In [10]:
q=QuantumCircuit(3)

q.x(1)

q.append(f,[0,1,2])

q.draw()

┌──────────┐
q_0: ─────┤0         ├
     ┌───┐│          │
q_1: ┤ X ├┤1 Unitary ├
     └───┘│          │
q_2: ─────┤2         ├
          └──────────┘

In [11]:
q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

<IPython.core.display.Latex object>

In [12]:
#Comprobamos para |110>

q=QuantumCircuit(3,2)

q.x(0)
q.x(1)

q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

<IPython.core.display.Latex object>

In [13]:
q=QuantumCircuit(3,2)

q.x(0)
q.x(1)

q.append(f,[0,1,2])

q.draw()

┌───┐┌──────────┐
q_0: ┤ X ├┤0         ├
     ├───┤│          │
q_1: ┤ X ├┤1 Unitary ├
     └───┘│          │
q_2: ─────┤2         ├
          └──────────┘
c: 2/═════════════════

In [14]:
q.save_statevector()
qobj = assemble(q)
final_state = svsim.run(qobj).result().get_statevector()
array_to_latex(final_state, prefix="\\text{Statevector} = ")

# Que devuelve |010>, cuando debería de se |110>

<IPython.core.display.Latex object>

In [ ]:
"""
Es evidente que los resultados de f no son los correctos, esto se debe a que qiskit analiza las filas de 
la matriz de una manera distinta, y no siguiento el formato usual como el del libro que hemos utilizado.

Veamos el ejemplo solo en la columna:

Natural   ->  Qiskit

1 |000>       1 |000>
2 |001>       5 |100>
3 |010>       3 |010>
4 |011>       7 |110>
5 |100>       2 |001>
6 |101>       6 |101>
7 |110>       4 |011>
8 |111>       8 |111>

Esto se puede comprobar simplemente llamando al statevector de cada estado.

Por esta razón la matriz de f no funcionaba bien, es debido a que la base está ordenada de otra manera.
Es decir, deberíamos reordenar la matriz para que represente lo que nosotros queríamos.

"""

In [15]:
# Cambiamos la matriz de f a g, con el cambio de orden de la base

f=Operator([[0,1,0,0,0,0,0,0],
            [1,0,0,0,0,0,0,0],
            [0,0,0,1,0,0,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,0,0,1,0,0],
            [0,0,0,0,0,0,1,0],
            [0,0,0,0,0,0,0,1]])

g=Operator([[0,1,0,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,1,0,0,0,0],
            [0,0,0,0,0,0,1,0],
            [1,0,0,0,0,0,0,0],
            [0,0,0,0,0,1,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,0,0,0,1]])

In [16]:
q=QuantumCircuit(3,2)

q.x(2)
q.barrier()
for i in range(3):
    q.h(i)
q.barrier()
q.append(f,[0,1,2])
q.barrier()
for i in range(0,2):
    q.h(i)
q.barrier()
q.measure(0,0)
q.measure(1,1)
q.draw()

░ ┌───┐ ░ ┌──────────┐ ░ ┌───┐ ░ ┌─┐   
q_0: ──────░─┤ H ├─░─┤0         ├─░─┤ H ├─░─┤M├───
           ░ ├───┤ ░ │          │ ░ ├───┤ ░ └╥┘┌─┐
q_1: ──────░─┤ H ├─░─┤1 Unitary ├─░─┤ H ├─░──╫─┤M├
     ┌───┐ ░ ├───┤ ░ │          │ ░ └───┘ ░  ║ └╥┘
q_2: ┤ X ├─░─┤ H ├─░─┤2         ├─░───────░──╫──╫─
     └───┘ ░ └───┘ ░ └──────────┘ ░       ░  ║  ║ 
c: 2/════════════════════════════════════════╩══╩═
                                             0  1

In [17]:
compiled_circuit_c= transpile(q, simulator)
job_c = simulator.run(compiled_circuit_c, shots=100)
result_c = job_c.result()
counts_c = result_c.get_counts(compiled_circuit_c)
print("dj_f:",counts_c)

# Esto quiere decir que es contante!, pero evidentemente es balanceada por definición. Esto se debe al problema
# de la base de qiskit. 

dj_f: {'00': 100}


In [18]:
#Probaremos ahora con g

q=QuantumCircuit(3,2)

q.x(2)
q.barrier()
for i in range(3):
    q.h(i)
q.barrier()
q.append(g,[0,1,2])
q.barrier()
for i in range(0,2):
    q.h(i)
q.barrier()
q.measure(0,0)
q.measure(1,1)
q.draw()

░ ┌───┐ ░ ┌──────────┐ ░ ┌───┐ ░ ┌─┐   
q_0: ──────░─┤ H ├─░─┤0         ├─░─┤ H ├─░─┤M├───
           ░ ├───┤ ░ │          │ ░ ├───┤ ░ └╥┘┌─┐
q_1: ──────░─┤ H ├─░─┤1 Unitary ├─░─┤ H ├─░──╫─┤M├
     ┌───┐ ░ ├───┤ ░ │          │ ░ └───┘ ░  ║ └╥┘
q_2: ┤ X ├─░─┤ H ├─░─┤2         ├─░───────░──╫──╫─
     └───┘ ░ └───┘ ░ └──────────┘ ░       ░  ║  ║ 
c: 2/════════════════════════════════════════╩══╩═
                                             0  1

In [19]:
compiled_circuit_c= transpile(q, simulator)
job_c = simulator.run(compiled_circuit_c, shots=100)
result_c = job_c.result()
counts_c = result_c.get_counts(compiled_circuit_c)
print("dj_g:",counts_c)

dj_g: {'01': 100}


In [20]:
# Veamos que ocurre si invertimos el algoritmo sobre los qubits, pero manteniendo f

q=QuantumCircuit(3,2)

q.x(0)
q.barrier()
for i in range(3):
    q.h(i)
q.barrier()
q.append(f,[0,1,2])
q.barrier()
for i in range(1,3):
    q.h(i)
q.barrier()
q.measure(1,1)
q.measure(2,0)
q.draw()

┌───┐ ░ ┌───┐ ░ ┌──────────┐ ░       ░       
q_0: ┤ X ├─░─┤ H ├─░─┤0         ├─░───────░───────
     └───┘ ░ ├───┤ ░ │          │ ░ ┌───┐ ░ ┌─┐   
q_1: ──────░─┤ H ├─░─┤1 Unitary ├─░─┤ H ├─░─┤M├───
           ░ ├───┤ ░ │          │ ░ ├───┤ ░ └╥┘┌─┐
q_2: ──────░─┤ H ├─░─┤2         ├─░─┤ H ├─░──╫─┤M├
           ░ └───┘ ░ └──────────┘ ░ └───┘ ░  ║ └╥┘
c: 2/════════════════════════════════════════╩══╩═
                                             1  0

In [21]:
compiled_circuit_c= transpile(q, simulator)
job_c = simulator.run(compiled_circuit_c, shots=100)
result_c = job_c.result()
counts_c = result_c.get_counts(compiled_circuit_c)
print("dj'_f:",counts_c)

# Obtenemos exactamente el mismo resultado que usando g con el algoritmo directo.

dj'_f: {'01': 100}


In [ ]:
# Podemos concluir además, que hemos conseguido una función f balanceada talque el resultado del algoritmo de
# Deustch-Jozsa no da el ket |11..1>, como ocurre con todos los oraculos balanceados que preparamos con las
# indicaciones que nos enseña qiskit.